<a href="https://colab.research.google.com/github/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/main/Logic/SensorVisualLogicTEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys

try:
    # Clone the GitHub repository if not already present
    if not os.path.exists("/content/Project-Pheonix"):
        !git clone https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix.git /content/Project-Pheonix

    # Change directory to project root
    %cd /content/Project-Pheonix

    # Checkout the 'main' branch
    !git fetch origin -q
    !git checkout main -q

    # Add project directory to Python path
    sys.path.append("/content/Project-Pheonix/Logic")

    #Install required Python packages
    %pip install -q firebase
    %pip install -q -U gradio
    %pip install -q plotly
    %pip install -q pandas
    %pip install -q importnb

    print("✅ Setup completed successfully.")
    from importnb import Notebook
    with Notebook():
        import CloudDB as dbService
    import gradio as gr
    import time
    from datetime import datetime
    from firebase import firebase
    import plotly.graph_objects as go

except Exception as e:
    print("❌ Setup failed:", str(e))

# Clear installation output for cleaner display
#from IPython.display import clear_output
#clear_output()

In [ ]:
#Fetches data from the database based on the enviorment
def fetch_data(path):
    data = dbService.get_from_db(f'/FakeData/{path}') or {}
    if data:
        keys = list(data.keys())
        values = list(data.values())
        readable_times = [datetime.utcfromtimestamp(int(ts)).strftime('%H:%M:%S') for ts in keys]
        dates = [datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d') for ts in keys]
    else:
        keys, values, readable_times, dates = [], [], [], []
    return keys, values, readable_times, dates

data_keys_indoor, data_values_indoor, times_indoor, dates_indoor = fetch_data("indoor")
data_keys_outdoor, data_values_outdoor, times_outdoor, _ = fetch_data("outdoor")


enviorment = ['indoor','outdoor']
sensors = {'indoor':['Distance','Temperature','Humidity','Pressure'] , 'outdoor':['DLIGHT','Temperature','Humidity','Pressure']}
sensor_units_map = {'Temperature': '°C','Humidity': '%','Pressure': 'Pa','Distance': 'mm','DLIGHT': 'lx'}
#create and array for the dates and inital hours

date_hour_set = { datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d,%H') for ts in data_keys_indoor}
dates = sorted(set(date.split(',')[0] for date in date_hour_set))
initial_hours = sorted(set( datetime.utcfromtimestamp(int(ts)).strftime('%H') for ts in data_keys_indoor if datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d') == dates[0]))

#updates Dropbox values
def rs_change(rs):
  return gr.update(choices=sensors[rs], value=sensors[rs][0])

#changes the hours in the dropbox according to the data and the selected date
def date_change(selected_date):
    hours = sorted(set(
        datetime.utcfromtimestamp(int(ts)).strftime('%H')
        for ts in data_keys_indoor
        if datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d') == selected_date
    ))
    return gr.update(choices=hours, value=hours[0] if hours else None)

#gets all the sensor's data in the selected hour and date and returns the average value of the sensor
def get_time_and_date(data_keys, data_values, selected_date, selected_hour, name):
    filtered = [
        (datetime.utcfromtimestamp(int(ts)).strftime('%H:%M:%S'), value.get(name, 0))
        for ts, value in zip(data_keys, data_values)
        if datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d') == selected_date
        and datetime.utcfromtimestamp(int(ts)).strftime('%H') == selected_hour
        and value.get(name, 0) != 0
    ]
    times, values = zip(*filtered) if filtered else ([], [])
    clean_values = [v for v in values if v is not None]
    avg = round(sum(clean_values) / len(clean_values), 2) if clean_values else 0
    return list(times), list(values), avg

# Data visualization
def plot_graph(place, name, date, hour):
    data = {
        'indoor': (data_keys_indoor, data_values_indoor),
        'outdoor': (data_keys_outdoor, data_values_outdoor)
    }.get(place)

    if not all([place, name, date, hour]):
        fig = go.Figure().update_layout(
            title="⚠️ Missing selection: Please choose environment, sensor, date, and hour."
        )
        return fig, "Error: Missing input."

    if not data:
        fig = go.Figure().update_layout(title="No data available")
        return fig, "No average"

    try:
        keys, values = data
        times, values, avg = get_time_and_date(keys, values, date, hour, name)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=times, y=values, mode='lines', name=name))
        fig.update_layout(
            title=f'Sensor {name} Over Time',
            xaxis_title='Time',
            yaxis_title=f"{name} ({sensor_units_map.get(name, '')})"
        )
        return fig, f"Average: {avg} {sensor_units_map.get(name, '')}"
    except Exception as e:
        fig = go.Figure().update_layout(title=f'Error: {str(e)}')
        return fig, "Error occurred"